In [ ]:
# Basic libraries import
import numpy as np
import pandas as pd
import seaborn as sns

import os

# Plotting
#%matplotlib
%matplotlib notebook
#%matplotlib inline

sns.set_context("paper", font_scale=1)

# Load Data
**data source**  
https://nsidc.org/data/seaice_index/archives.html

In [ ]:
def load_data(filename):
    dirpath = os.path.join(os.path.pardir, 'data')
    df = pd.read_csv(os.path.join(dirpath, filename), comment='#').drop(['Missing', 'Source Data'], axis=1)
    # Discard day and keep the average
    df = df.groupby(['Year', 'Month'])[['Extent']].mean().reset_index()
    year_df = df.groupby('Year', as_index=False)[['Extent']].mean()
    return df, year_df

In [ ]:
n_data, n_yearly_data = load_data('N_seaice_extent_daily_v2.1.csv')
s_data, s_yearly_data = load_data('S_seaice_extent_daily_v2.1.csv')

In [ ]:
n_yearly_data.head()

# Animations

In [ ]:
# square animation
from matplotlib import pyplot as plt, animation
import matplotlib.patches as patches
fig, ax = plt.subplots()
ax.set_xlim([0,20])
ax.set_ylim([5,20])
rectangle = patches.Rectangle((0, 0), 0, 0)
epoch_text = sns.plt.text(0, 19, "Start")
note = sns.plt.text(0, 15, "Start")
ax.set_title("Daily Sea Ice Extent Northern Hemisphere")
ax.set_ylabel('Extent (10^6 sq km)')
#sns.plt.show()
y_rectangle = patches.Rectangle((0, 0), 0, 0)
ax.add_patch(rectangle)
ax.add_patch(y_rectangle)

def animate(i):
    entry = df.iloc[i,:]
    year = int(entry['Year'])
    prev_year = int(df.iloc[i-1]['Year'])
    if year != prev_year:
        average = g_df.loc[prev_year]['Extent']
        y_rectangle = patches.Rectangle((0, 0), average, average, fill=False)
        ax.add_patch(y_rectangle)
        note.set_text('{} average'.format(prev_year))
        note.set_position((15, average-1))
    rectangle.set_width(entry['Extent'])
    rectangle.set_height(entry['Extent'])
    epoch_text.set_text("Extent for {} - {}".format(int(entry['Year']), int(entry['Month'])))
    return line,

ani = animation.FuncAnimation(fig, animate, np.arange(0, len(df)), interval=100)

In [ ]:
# line animation
from matplotlib import pyplot as plt, animation
fig, ax = plt.subplots()
ax.set_xlim([0,20])
ax.set_ylim([5,20])
line, = sns.plt.plot([0, 20], [0, 0], 'k-')
epoch_text = sns.plt.text(0, 19, "Start")
note = sns.plt.text(0, 15, "Start")
ax.set_title("Daily Sea Ice Extent Northern Hemisphere")
ax.set_ylabel('Extent (10^6 sq km)')
#sns.plt.show()
y_line = None

def animate(i):
    entry = df.iloc[i,:]
    year = int(entry['Year'])
    prev_year = int(df.iloc[i-1]['Year'])
    if year != prev_year:
        average = g_df.loc[prev_year]['Extent']
        global y_line
        if y_line is not None:
            y_line.set_color('gray')
        y_line, = sns.plt.plot([0, 20], [average, average], color='blue')
        note.set_text('{} average'.format(prev_year))
        note.set_position((15, average-1))
        #ax.annotate('{} average'.format(prev_year), xy=(15, average))
    #print(entry['Extent'])
    #sns.plt.plot([0, 20], [entry['Extent'], entry['Extent']], 'k-')
    line.set_ydata([entry['Extent'], entry['Extent']])
    epoch_text.set_text("Extent for {} - {}".format(int(entry['Year']), int(entry['Month'])))
    #epoch_text.set_text("{}".format(entry['Year']))
    return line,

#ani = animation.FuncAnimation(fig, animate, np.arange(0, len(df)), interval=100, blit=False, save_count=len(df))

In [ ]:
#FFwriter = animation.FFMpegWriter(fps=30)
#animation.FuncAnimation(fig, animate, np.arange(0, len(df))).save('basic_animation.gif', writer ='ffmpeg')

In [ ]:
animation.FuncAnimation(fig, animate, np.arange(0, len(df))).save('basic_animation.mp4', writer=animation.FFMpegFileWriter(fps=30))

# Plotting

In [ ]:
# combine thickness data (south + north)
n_yearly_data['Emisphere'] = 'n'
s_yearly_data['Emisphere'] = 's'
n_yearly_data['1979_year_diff'] = n_yearly_data['Extent'] - n_yearly_data[n_yearly_data['Year']==1979]['Extent'].iloc[0]
s_yearly_data['1979_year_diff'] = s_yearly_data['Extent'] - s_yearly_data[s_yearly_data['Year']==1979]['Extent'].iloc[0]
year_data = pd.concat([n_yearly_data, s_yearly_data], ignore_index=True)
year_data['Prev_year_diff'] = year_data['Extent'] - year_data['Extent'].shift()
year_data = year_data[(year_data['Year']!=2017) & (year_data['Year']!=1978)]
year_data.head()

In [ ]:
year_data.to_csv(os.path.join(os.path.pardir, 'data', 'nsidc_seaice.csv'), index_label=False)
year_data.to_json(os.path.join(os.path.pardir, 'data', 'nsidc_seaice.json'), orient = 'records')

## Seaborn

In [ ]:
g = sns.FacetGrid(year_data, row="Emisphere")
g.fig.suptitle('Sea Ice Extent (10^6 sq km)')
g = g.map(sns.pointplot, 'Year', 'Extent')
sns.plt.show()

In [ ]:
g = sns.FacetGrid(year_data, row="Emisphere")
g.fig.suptitle('Sea Ice Extent - Difference from previous year (10^6 sq km)')
g = g.map(plt.bar, 'Year', 'Prev_year_diff')
sns.plt.show()

In [ ]:
g = sns.FacetGrid(year_data, row="Emisphere")
g.fig.suptitle('Sea Ice Extent - Difference from 1979 (10^6 sq km)')
g = g.map(plt.bar, 'Year', '1979_year_diff')
sns.plt.show()

## Matplotlib

In [ ]:
fig, axes = sns.plt.subplots(1,2)
axes[0].bar(n_dif.index.values, n_dif.values)
axes[0].set_title('North Sea Ice Extent')
axes[1].bar(s_dif.index.values, s_dif.values)
axes[1].set_title('South Sea Ice Extent')
sns.plt.show()

In [ ]:
fig, axes = sns.plt.subplots(1,2)
axes[0].bar(n_dif.index.values, n_dif.values)
axes[0].set_title('North Sea Ice Extent')
axes[1].bar(s_dif.index.values, s_dif.values)
axes[1].set_title('South Sea Ice Extent')
sns.plt.show()

## Plotly

In [ ]:
import plotly.plotly as py
py.sign_in('username', 'password')

In [ ]:
import cufflinks as cf
cf.go_online()

In [ ]:
stacked_data = year_data[['Year', 'Extent', 'Emisphere']].set_index(['Year', 'Emisphere']).unstack(level=1)
stacked_data.columns = ['North Hemisphere', 'South Hemisphere']
stacked_data.head()

In [ ]:
stacked_data.iplot(title="Sea Ice Extent", yTitle = 'Sea Ice Extent (10^6 sq km)')

In [ ]:
stacked_data = year_data[year_data['Year']!=1979][['Year', 'Prev_year_diff', 'Emisphere']].set_index(['Year', 'Emisphere']).unstack(level=1)
stacked_data.columns = ['North Hemisphere', 'South Hemisphere']
stacked_data.iplot(title="Sea Ice Extent - Difference from Previous Year", kind='bar', yTitle = 'Difference from previous year (10^6 sq km)')

In [ ]:
stacked_data = year_data[['Year', '1979_year_diff', 'Emisphere']].set_index(['Year', 'Emisphere']).unstack(level=1)
stacked_data.columns = ['North Hemisphere', 'South Hemisphere']
stacked_data.iplot(title='Sea Ice Extent - Difference from 1979', kind='bar', yTitle = 'Difference from 1979 (10^6 sq km)')